## model

- model.modules()
- model.children

### 创建网络模型-标准流程

pytorch在搭建网络模型时，最标准的模式是首先是创建一个继承`nn.Module`的类，并且在类的初始化函数(\_\_init\_\_)中定义layer成员变量(注意，layer变量也可以是另外一个Module实例化的对象)，在前向传播函数(forward)中确定如何使用layer变量组合成一个网络模型，即相当于规定了输入经过层层处理并最终得到输出结果的`流程图`!

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [28]:
class Features(nn.Module):
    def __init__(self):
        super(Features, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, (5,5))
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
    def forward(x):
        x = self.conv1(x)
        x = self.pool(F.ReLU(x))
        x = self.conv2(x)
        x = self.pool(F.ReLU(x))
        return x
    
class Classifer(nn.Module):
    def __init__(self):
        super(Classifer, self).__init__()
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(64, 10)
    def forward(x):
        x = x.view(-1, 16*5*5)
        x = F.ReLU(self.fc1(x))
        x = F.ReLU(self.fc2(x))
        x = self.fc3(x)
        return x
    
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.features = Features()
        self.classifer = Classifer()
        
    def forward(x):
        x = self.features(x)
        x = self.classifer(x)
        return x

In [29]:
net = MyNet()
print(net)

MyNet(
  (features): Features(
    (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  )
  (classifer): Classifer(
    (fc1): Linear(in_features=400, out_features=120, bias=True)
    (fc2): Linear(in_features=120, out_features=84, bias=True)
    (fc3): Linear(in_features=64, out_features=10, bias=True)
  )
)


注意观察pytorch打印出的模型结果，对更好的使用pytorch内置的模型有帮助，其格式是：
```
Net(
    (subnet1_obj):SubNet1_ClassName(
        (layer1_obj):Layer1_ClassName(...)
        (layer2_obj):Layer2_ClassName(...)
    ...
    )
    (subnet2_obj):SubNet2_ClassName(
        (layer1_obj):Layer1_ClassName(...)
        (layer2_obj):Layer1_ClassName(...)
    ...
    )
    ...
)
```

### 访问网络模型的类成员
通过print网络模型对象，可以观察其网络结构，进而访问各layer对应的成员变量

In [27]:
net.features

Features(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
)

In [37]:
net.features.conv1

Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))

### 创建网络模型-Sequential
使用Sequential创建网络模型有两种方式：
- 创建Sequential对象时同时传入layer;
- 先创建Sequential对象，使用add_module()函数添加layer;

In [42]:
class Features(nn.Module):
    def __init__(self):
        super(Features, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 6, (5,5)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6, 16, (5,5)),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
    def forward(x):
        x = self.conv(x)
        return x
    
class Classifer(nn.Module):
    def __init__(self):
        super(Classifer, self).__init__()
        self.fc = nn.Sequential()
        self.fc.add_module("fc1", nn.Linear(16*5*5, 120))
        self.fc.add_module("ReLU1", nn.ReLU())
        self.fc.add_module("fc2", nn.Linear(120, 84))
        self.fc.add_module("ReLU2", nn.ReLU())
        self.fc.add_module("fc3", nn.Linear(64, 10))
        
    def forward(x):
        x = x.view(-1, 16*5*5)
        x = self.fc(x)
        return x
    
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.features = Features()
        self.classifer = Classifer()
        
    def forward(x):
        x = self.features(x)
        x = self.classifer(x)
        return x

In [43]:
net = MyNet()
print(net)

MyNet(
  (features): Features(
    (conv): Sequential(
      (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
      (4): ReLU()
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (classifer): Classifer(
    (fc): Sequential(
      (fc1): Linear(in_features=400, out_features=120, bias=True)
      (ReLU1): ReLU()
      (fc2): Linear(in_features=120, out_features=84, bias=True)
      (ReLU2): ReLU()
      (fc3): Linear(in_features=64, out_features=10, bias=True)
    )
  )
)


### 访问网络模型的类成员
对于使用Sequential第一种方法创建的网络，可以通过索引的方式访问Sequential内的各layer

In [44]:
net.features.conv[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))

对于使用Sequential第二种方法创建的网络，可以通过成员变量方式访问Sequential内的各layer

In [45]:
net.classifer.fc.fc1

Linear(in_features=400, out_features=120, bias=True)

## alexnet

In [30]:
import torchvision.models as models

In [2]:
alexnet = models.alexnet()
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [33]:
#
alexnet.features[0]

Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

In [3]:
alexnet.named_modules

<bound method Module.named_modules of AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features

In [5]:
list(alexnet.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
   (1): ReLU(inplace=True)
   (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
   (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   (4): ReLU(inplace=True)
   (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
   (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (7): ReLU(inplace=True)
   (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (9): ReLU(inplace=True)
   (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (11): ReLU(inplace=True)
   (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
 ),
 AdaptiveAvgPool2d(output_size=(6, 6)),
 Sequential(
   (0): Dropout(p=0.5, inplace=False)
   (1): Linear(in_features=9216, out_features=4096, bias=True)
   (2): ReLU(inplace=True)
   (3): Dropout(p=0.5, inplace=False)

In [6]:
len(list(alexnet.children()))

3

In [14]:
list(alexnet.children())[0]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)

## resnet50

In [15]:
resnet = models.resnet50()
resnet.modules

<bound method Module.modules of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_si

In [17]:
type(resnet.layer1)

torch.nn.modules.container.Sequential